In [13]:
import nfl_data_py as nfl
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

Getting NFL data from the last 9 years

In [2]:
years = list(range(2015,2025))

In [3]:
nfl_main_df = nfl.import_weekly_data(years, columns = list(nfl.see_weekly_cols()), downcast=True)

Downcasting floats.


In [4]:
schedule_df = nfl.import_schedules(years)

In [5]:
team_opponent_df_1 = schedule_df[["week","season","away_team","home_team","game_id"]].rename(columns={"away_team":"recent_team","home_team":"opponent"})
team_opponent_df_2 = schedule_df[["week","season","away_team","home_team","game_id"]].rename(columns={"away_team":"opponent","home_team":"recent_team"})

team_opponent_df = pd.concat([team_opponent_df_1,team_opponent_df_2])

In [6]:
nfl_main_df = nfl_main_df.merge(team_opponent_df,on=["week","season","recent_team"],how="left")

In [7]:
nfl_main_df = nfl_main_df.rename(columns={"recent_team":"team"})

In [11]:
# Function to respond to dropdown selection and return plot
def game_dashboard(week, season, team, stats): 
    with output:
        # Clear previous plot
        clear_output(wait=True)

        # Filter data based on week, season, and team
        week_df = nfl_main_df[(nfl_main_df["week"] == week) & (nfl_main_df["season"] == season)]
        if week_df.empty:
            print("No data for the selected filters.")
            return

        try:
            
            
            game_id = week_df[week_df["team"] == team]["game_id"].values[0]
            game_df = nfl_main_df[nfl_main_df["game_id"] == game_id]
            
            # Create the plot
            
            # Determine grid size based on number of stats
            num_stats = len(stats)
            ncols = 2
            nrows = int(np.ceil(num_stats / ncols))
            
            
            f, axes = plt.subplots(nrows, ncols, figsize=(15, 5 * nrows))
            sns.set_style("darkgrid")
            
            # Flatten axes array for easier iteration
            axes = axes.flatten()

            # Plot each statistic
            for i, s in enumerate(stats):
                filtered_data = game_df[game_df[s] > 0].sort_values(s,ascending=False)
                if not filtered_data.empty:
                    sns.barplot(x="player_display_name", y=s, hue="team",
                                data=filtered_data, ax=axes[i])
                    axes[i].set_title(s)
                else:
                    axes[i].text(0.5, 0.5, 'No Data', horizontalalignment='center',
                                 verticalalignment='center', fontsize=12, color='red')
                plt.setp(axes[i].get_xticklabels(), rotation=70, ha='center')

            # Turn off any unused subplots
            for j in range(num_stats, len(axes)):
                axes[j].axis('off')
                
            for ax in axes.flat:
                plt.setp(ax.get_xticklabels(), rotation=70)

            # Adjust space between rows of the subplot
            plt.subplots_adjust(hspace=0.55, wspace=0.3)
            plt.show()

        except IndexError:
            print("No matching game data for the selected week, season, and team.")

In [14]:
# Create four dropdown widgets
dropdown1 = widgets.Dropdown(
    options=list(nfl_main_df["week"].sort_values().unique()),
    value=1,
    description='Week: ',
)

dropdown2 = widgets.Dropdown(
    options=list(nfl_main_df["season"].sort_values().unique()),
    value=2024,
    description='Season: ',
)

dropdown3 = widgets.Dropdown(
    options=list(nfl_main_df["team"].sort_values().unique()),
    value='PHI',
    description='Team: ',
)

dropdown4 = widgets.SelectMultiple(
    options=['completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs'],
    value=['completions', 'passing_tds', 'receptions', 'receiving_tds'],  # Default selection
    description='Stats:',
)

# Output widget to control where the plot will be displayed
output = widgets.Output()



# Function to update the plot based on dropdown values
def update_plot(*args):
    game_dashboard(dropdown1.value, dropdown2.value, dropdown3.value, list(dropdown4.value))

# Observe dropdown value changes
dropdown1.observe(update_plot, names='value')
dropdown2.observe(update_plot, names='value')
dropdown3.observe(update_plot, names='value')
dropdown4.observe(update_plot, names='value')

# Display dropdowns and output
display(dropdown1, dropdown2, dropdown3, dropdown4, output)

# Call the plot function initially
update_plot()

Dropdown(description='Week: ', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,…

Dropdown(description='Season: ', index=9, options=(2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024)…

Dropdown(description='Team: ', index=25, options=('ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL…

SelectMultiple(description='Stats:', index=(0, 3, 24, 27), options=('completions', 'attempts', 'passing_yards'…

Output()